In [ ]:
using JuMP
using CPLEX
using Distributions
using LinearAlgebra
using Statistics
using Dates
using DataFrames
using SDDP
using Plots
import CSV
using JSON
try
    using Revise
catch e
    @warn "Error initializing Revise" exception=(e, catch_backtrace())
end

### Bidding Model  
At the Electricity Markets, Participants have to communicate increasing bidding curves of price-volume pairs before market clearing.  
Depending on the market clearing price, the delivery of each participant becomes known  
We approximate this relation by using linear interpolation of volumes and presetting price Points, for example based on probabilities.

$$
y_t = \frac{c_t - P_{t,i} }{P_{t,i+1} - P_{t,i}} \cdot x_{i,t} + \frac{P_{t,i+1} - c_t }{P_{t,i+1} - P{t,i}} \cdot x_{i+1,t}, \qquad \text{if} \; P_{t,i} \leq c_t \leq P_{t,i+1}
$$

The Volumes have to be in increasing order:

$$
x_{i,t} \leq x_{i+1,t}
$$

In [ ]:
Omega = [10.0, 20.0, 30.0]
P = [1/length(Omega) for om in Omega]

# One Price Point around each uncertain Price
PPoints = [5.0, 15, 25, 35]

In [170]:
function subproblem_builder(subproblem::Model, node::Int64)
    @variable(subproblem, 0 <= x[i = 1:length(PPoints)] <= 50, SDDP.State, initial_value=0)
    @variable(subproblem, 0 <= W <= 100, SDDP.State, initial_value = 80)
    @variable(subproblem, y[i = 1:length(PPoints)-1] >= 0)
    @variable(subproblem, w[i = 1:length(PPoints)-1] >= 0)
    @variable(subproblem, balancing[i = 1:length(PPoints)-1] >= 0)
    @constraint(subproblem, clearing[i = 1:length(PPoints)-1], y[i] == 1* x[i].in +  1* x[i+1].in)
    @constraint(subproblem, increasing[i = 1:length(PPoints)-1], x[i].out <= x[i+1].out)
    if node == 1
        @stageobjective(subproblem, 0)
        @constraint(subproblem, W.in == W.out)
    else
        @constraint(subproblem, production[i = 1:length(PPoints)-1], y[i] == w[i] + balancing[i])
        @constraint(subproblem, balance, W.out == W.in - sum(w[i] for i in 1:length(PPoints)-1))
        SDDP.parameterize(subproblem, Omega, P) do price
            for i in 1:length(PPoints)-1
                if (price >= PPoints[i]) && (price <= PPoints[i+1])
                    set_normalized_coefficient(clearing[i], x[i].in, -((price - PPoints[i])/(PPoints[i+1] - PPoints[i])))
                    set_normalized_coefficient(clearing[i], x[i+1].in, -((PPoints[i+1] - price)/(PPoints[i+1] - PPoints[i])))
                    set_normalized_coefficient(production[i], w[i], -1)
                    set_normalized_coefficient(production[i], balancing[i], -1)
                    set_normalized_coefficient(production[i], y[i], 1)
                    @stageobjective(subproblem, price * (y[i] - balancing[i]))
                else
                    set_normalized_coefficient(balance, w[i], 0)
                    set_normalized_coefficient(production[i], w[i], 0)
                    set_normalized_coefficient(production[i], balancing[i], 0)
                    set_normalized_coefficient(production[i], y[i], 0)
                    set_normalized_coefficient(clearing[i], x[i].in, 0)
                    set_normalized_coefficient(clearing[i], x[i+1].in, 0)
                end
            end
        end
    end
    return
end

subproblem_builder (generic function with 1 method)

In [171]:
model = SDDP.LinearPolicyGraph(
    subproblem_builder;
    stages = 2,
    sense = :Max,
    upper_bound = 1e5,
    optimizer = CPLEX.Optimizer
)

A policy graph with 2 nodes.
 Node indices: 1, 2


In [172]:
det_equiv = SDDP.deterministic_equivalent(model, CPLEX.Optimizer)
JuMP.write_to_file(det_equiv, "det_equiv.lp")
optimize!(det_equiv)
solution_summary(det_equiv; verbose=true)

CPLEX Error  3003: Not a mixed-integer problem.
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
Tried aggregator 1 time.
LP Presolve eliminated 42 rows and 70 columns.
Aggregator did 10 substitutions.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.03 ticks)


* Solver : CPLEX

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 0.00000e+00
  Objective bound    : 0.00000e+00
  Dual objective value : 0.00000e+00
  Primal solution :
    W_in#1 : 8.00000e+01
    W_in#2 : 8.00000e+01
    W_out#1 : 8.00000e+01
    W_out#2 : 8.00000e+01
    _[20]#1 : 1.00000e+05
    _[20]#2 : 0.00000e+00
    balancing[1]#1 : 0.00000e+00
    balancing[1]#2 : 0.00000e+00
    balancing[2]#1 : 0.00000e+00
    balancing[2]#2 : 0.00000e+00
    balancing[3]#1 : 0.00000e+00
    balancing[3]#2 : 5.00000e+01
    w[1]#1 : 0.00000e+00
    w[1]#2 : 0.00000e+00
    w[2]#1 : 0.00000e+00
    w[2]#2 : 0.00000e+00
    w[3]#1 : 0.00000e+00
    w[3]#2 : 0.00000e+00
    x[1]_in#1 : 0.00000e+00
    x[1]_in#2 : -0.00000e+00
    x[1]_out#1 : 0.00000e+00
    x[1]_out#2 : 0.00000e+00
    x[2]_in#1 : 0.00000e

In [173]:
SDDP.train(model; iteration_limit = 100)

-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-23
-------------------------------------------------------------------
problem
  nodes           : 2
  state variables : 5
  scenarios       : 3.00000e+00
  existing cuts   : false
options
  solver          : serial mode
  risk measure    : SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                             : [20, 20]
  AffExpr in MOI.EqualTo{Float64}         : [4, 7]
  AffExpr in MOI.LessThan{Float64}        : [3, 3]
  VariableRef in MOI.GreaterThan{Float64} : [14, 15]
  VariableRef in MOI.LessThan{Float64}    : [6, 6]
numerical stability report
  matrix range     [5e-01, 1e+00]
  objective range  [1e+00, 3e+01]
  bounds range     [5e+01, 1e+05]
  rhs range        [0e+00, 0e+00]
-------------------------------------------------------------------
 iteration    simulation      bound        time (s)     solv


        20   1.500000e+03  1.000000e+03  3.299999e-02       126   1
-------------------------------------------------------------------
status         : simulation_stopping
total time (s) : 3.299999e-02
total solves   : 126
best bound     :  1.000000e+03
simulation ci  :  9.250000e+02 ± 1.780968e+02
numeric issues : 0
-------------------------------------------------------------------



In [174]:
simulations = SDDP.simulate(
    # The trained model to simulate.
    model,
    # The number of replications.
    100,
    # A list of names to record the values of.
    [:x, :y, :w, :balancing],
)

replication = 28
stage = 2
println("Bidding Curve Values: ", simulations[replication][stage][:x])
println("Cleared Volume: ", simulations[replication][stage][:y])
println("Own Producion: ", simulations[replication][stage][:w])
println("Purchases: ", simulations[replication][stage][:balancing])

Bidding Curve Values: SDDP.State{Float64}[SDDP.State{Float64}(50.0, 0.0), SDDP.State{Float64}(50.0, 0.0), SDDP.State{Float64}(50.0, 50.0), SDDP.State{Float64}(50.0, 50.0)]
Cleared Volume: [50.0, 0.0, 0.0]
Own Producion: [50.0, 0.0, 0.0]
Purchases: [0.0, 0.0, 0.0]


In [175]:
plt = SDDP.SpaghettiPlot(simulations)

A spaghetti plot with 100 scenarios and 2 stages.

In [176]:
sbproblem = JuMP.read_from_file("subproblem_2.mof.json")
println(sbproblem)

Max 30 y[3] - 30 balancing[3] + x20
Subject to
 clearing[1] : y[1] == 0
 clearing[2] : y[2] == 0
 clearing[3] : -0.5 x[3]_in - 0.5 x[4]_in + y[3] == 0
 production[1] : 0 == 0
 production[2] : 0 == 0
 production[3] : y[3] + w[3] + balancing[3] == 0
 balance : -W_in + W_out == 0
 increasing[1] : x[1]_out - x[2]_out <= 0
 increasing[2] : x[2]_out - x[3]_out <= 0
 increasing[3] : x[3]_out - x[4]_out <= 0
 x[1]_in == 0
 x[2]_in == 0
 x[3]_in == 50
 x[4]_in == 50
 W_in == 80
 x[1]_out >= 0
 x[2]_out >= 0
 x[3]_out >= 0
 x[4]_out >= 0
 W_out >= 0
 y[1] >= 0
 y[2] >= 0
 y[3] >= 0
 w[1] >= 0
 w[2] >= 0
 w[3] >= 0
 balancing[1] >= 0
 balancing[2] >= 0
 balancing[3] >= 0
 x20 >= 0
 x[1]_out <= 50
 x[2]_out <= 50
 x[3]_out <= 50
 x[4]_out <= 50
 W_out <= 100
 x20 <= 0

